In [2]:
import pandas as pd
import numpy as np
import os
import requests
import nltk as nltk
import nltk.corpus  
from nltk.text import Text
import pandas as pd
import re
import sys
import random
import spacy 
from spacy import displacy
from spacy.util import minibatch, compounding
from tqdm import tqdm
from pandarallel import pandarallel
import preprocessor as p
# import torch

In [2]:
pandarallel.initialize(nb_workers = os.cpu_count()-1,progress_bar=True)

INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
torch.cuda.is_available()

True

In [4]:
spacy.require_gpu()
nlp = spacy.load("en_core_web_sm")

In [8]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

In [ ]:
# news_df = pd.read_parquet('https://storage.googleapis.com/msca-bdp-data-open/news_final_project/news_final_project.parquet', engine='pyarrow')

In [7]:
news_df = pd.read_parquet('../data/news_df_companies.parquet', engine='pyarrow')

print(f'Sample contains {news_df.shape[0]:,.0f} news articles')
news_df.head(2)

Sample contains 200,332 news articles


,url,date,language,title,text,cleaned_text,companies,companies_nltk
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,Artificial intelligence improves parking effic...,"[ChinaGraduate, Chinas, Wang, People's Daily O...","[livesMicrosoft, People, AIpark Sky Eye, Infor..."
1,http://newsparliament.com/2020/02/27/children-...,2020-02-27,en,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,Children With Autism Saw Their Learning and So...,"[Science Robotics, Comments Author Recent Post...","[Highlights, All Rights Reserved, HRI, Arboris..."


In [ ]:
news_df.to_parquet("Vishal_NLP_data/news_df.parquet")

In [ ]:
print(news_df.iloc[555]['title'])

This Could Be The Next Multi-Billion AI Breakthrough


## Cleaning the Data

In [ ]:
nlp = spacy.load("en_core_web_sm",exclude=['tok2vec','lemmatizer'])

In [ ]:
# Function to simply remove all unnecessary whitespace, urls hashtags etc.
def text_processing(string):
    cleaned_string = p.clean(string)
    return cleaned_string

In [ ]:
news_df['cleaned_text'] = news_df['text'].parallel_apply(text_processing)

In [ ]:
news_df['cleaned_text'][0]

"Artificial intelligence improves parking efficiency in Chinese cities - People's Daily OnlineHomeChina PoliticsForeign AffairsOpinionsVideo: We Are ChinaBusinessMilitaryWorldSocietyCultureTravelScienceSportsPhotoLanguagesChineseJapaneseFrenchSpanishRussianArabicKoreanGermanPortugueseThursday, March , Home>>Artificial intelligence improves parking efficiency in Chinese citiesBy LiuShiyao (People's Daily) :16, March , Photo taken on July , , shows a sign for electronic toll collection (ETC) newly set up at a roadside parking space on Yangzhuang road, Shijingshan district, Beijing. Some urban areas of the city started to use ETC system for roadside parking spaces since July , . (Peoples Daily Online/Li Wenming)Thanks to the application of an artificial intelligence (AI)-empowered roadside electronic toll collection (ETC) system, Chinas capital city Beijing has seen significant improvement in the efficiency of parking fee collection, turnover of roadside parking spots, order in roadside p

In [7]:
spacy.require_cpu()

True

In [ ]:
title_entities = []
for doc in tqdm(nlp.pipe(
    texts = news_df['cleaned_text'],
    n_process=os.cpu_count()
), total=len(news_df)):
    ents = set()
    for ent in doc.ents:
        if ent.label_ in ['ORG']:
            ents.add(ent.text)
    title_entities.append(ents)

100%|██████████| 200332/200332 [25:40<00:00, 130.08it/s]


In [ ]:
news_df['companies'] = title_entities

In [ ]:
news_df

,url,date,language,title,text,cleaned_text,companies
0,http://en.people.cn/n3/2021/0318/c90000-9830122.html,2021-03-18,en,Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online,"\n\nArtificial intelligence improves parking efficiency in Chinese cities - People's Daily Online\n\nHome\nChina Politics\nForeign Affairs\nOpinions\nVideo: We Are China\nBusiness\nMilitary\nWorld\nSociety\nCulture\nTravel\nScience\nSports\nPhoto\n\nLanguages\n\nChinese\nJapanese\nFrench\nSpanish\nRussian\nArabic\nKorean\nGerman\nPortuguese\nThursday, March 18, 2021\nHome>>\n\t\t\nArtificial intelligence improves parking efficiency in Chinese cities\nBy Liu Shiyao (People's Daily) 09:16, Mar...","Artificial intelligence improves parking efficiency in Chinese cities - People's Daily OnlineHomeChina PoliticsForeign AffairsOpinionsVideo: We Are ChinaBusinessMilitaryWorldSocietyCultureTravelScienceSportsPhotoLanguagesChineseJapaneseFrenchSpanishRussianArabicKoreanGermanPortugueseThursday, March , Home>>Artificial intelligence improves parking efficiency in Chinese citiesBy LiuShiyao (People's Daily) :16, March , Photo taken on July , , shows a sign for electronic toll collection (ETC) ne...","{ChinaGraduate, Chinas, Wang, People's Daily Online, AIpark Sky Eye, Mobile, AIpark, ETC, ChinaGrain, Chinas Ministry of Industry and Information Technology, iFlytek, MIIT).Traditional, Peoples Daily Online/Li Wenming)Thanks, People's Daily, AI, LiuShiyao, the AIpark Sky Eye}"
1,http://newsparliament.com/2020/02/27/children-with-autism-saw-their-learning-and-social-skills-boosted-after-playing-with-this-ai-robot/,2020-02-27,en,Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot – News Parliament,"\nChildren With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot – News Parliament\n \n\nSkip to content\n\t\t\tThursday, February 27, 2020\t\t\n\nLatest:\n\n\nMansplaining in conferences: How can we get him to forestall?\n\n\nDrax power station to cease burning coal in March 2021\n\n\nCoronavirus Could Explode in the U.S. Overnight Like it Did in Italy\n\n\nCoronavirus: Dettol sales surge as markets fall again\n\n\nLevi Strauss marks the next phase in cor...","Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot News Parliament Skip to contentThursday, February , Latest:Mansplaining in conferences: How can we get him to forestall?Drax power station to cease burning coal in March Coronavirus Could Explode in the U.S. Overnight Like it Did in ItalyCoronavirus: Dettol sales surge as markets fall againLevi Strauss marks the next phase in corporate paid leave policiesNews ParliamentGet the Real Story NewsTe...","{Science Robotics, Comments Author Recent Posts, admin0 About Us News Parliament, HRI, New Comprehensive Report, Trending News Mansplaining, Drax, Matari, ASD, Alex Jones0 BusinessNewsServicesTechnology New Study Examines Global Water Treatment Services, CAD International (ArborCAD, The Dow Chemical, Dettol, PolicyContact Us News Sections BusinessEntertainmentHealthcareNewsServicesTechnologyUncategorizedWorld - News Parliament, Alex Jones0 BusinessServicesTechnology Massive Growth of Arboris..."
2,http://www.dataweek.co.za/12835r,2021-03-26,en,"Forget ML, AI and Industry 4.0 – obsolescence should be your focus - 26 February 2021 - Test & Rework Solutions - Dataweek","\n\nForget ML, AI and Industry 4.0 – obsolescence should be your focus - 26 February 2021 - Test & Rework Solutions - Dataweek\nHome\nAbout us\nBack issues / E-book / PDF\nEMP Handbook\nSubscribe\nAdvertise\n\nCategories\n\n▸ Editor's Choice\n▸ Multimedia, Videos\n▸ Analogue, Mixed Signal, LSI\n▸ Circuit & System Protection\n▸ Computer/Embedded Technology\n▸ Design Automation\n▸ DSP, Micros & Memory\n▸ Electronics Technology\n▸ Enclosures, Racks, Cabinets & Panel Products\n▸ Events\n▸ Interc...","Forget ML, AI and Industry obsolescence should b

In [ ]:
def find_company_nltk(string):
    tokens = nltk.word_tokenize(string)    
    entities = set()
    for chunk in nltk.ne_chunk(nltk.pos_tag(tokens), binary = False):
        if hasattr(chunk, 'label'):
            entity = ' '.join(c[0] for c in chunk)
            label = chunk.label()
            if label == 'ORGANIZATION':
                entities.add(entity)
    return(entities)

In [ ]:
news_df['companies_nltk'] = news_df['cleaned_text'].parallel_apply(find_company_nltk)

In [ ]:
news_df

,url,date,language,title,text,cleaned_text,companies,companies_nltk
0,http://en.people.cn/n3/2021/0318/c90000-9830122.html,2021-03-18,en,Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online,"\n\nArtificial intelligence improves parking efficiency in Chinese cities - People's Daily Online\n\nHome\nChina Politics\nForeign Affairs\nOpinions\nVideo: We Are China\nBusiness\nMilitary\nWorld\nSociety\nCulture\nTravel\nScience\nSports\nPhoto\n\nLanguages\n\nChinese\nJapanese\nFrench\nSpanish\nRussian\nArabic\nKorean\nGerman\nPortuguese\nThursday, March 18, 2021\nHome>>\n\t\t\nArtificial intelligence improves parking efficiency in Chinese cities\nBy Liu Shiyao (People's Daily) 09:16, Mar...","Artificial intelligence improves parking efficiency in Chinese cities - People's Daily OnlineHomeChina PoliticsForeign AffairsOpinionsVideo: We Are ChinaBusinessMilitaryWorldSocietyCultureTravelScienceSportsPhotoLanguagesChineseJapaneseFrenchSpanishRussianArabicKoreanGermanPortugueseThursday, March , Home>>Artificial intelligence improves parking efficiency in Chinese citiesBy LiuShiyao (People's Daily) :16, March , Photo taken on July , , shows a sign for electronic toll collection (ETC) ne...","{ChinaGraduate, Chinas, Wang, People's Daily Online, AIpark Sky Eye, Mobile, AIpark, ETC, ChinaGrain, Chinas Ministry of Industry and Information Technology, iFlytek, MIIT).Traditional, Peoples Daily Online/Li Wenming)Thanks, People's Daily, AI, LiuShiyao, the AIpark Sky Eye}","{livesMicrosoft, People, AIpark Sky Eye, Information Technology, US, Ministry, AIpark, ETC, All Rights Reserved, Yangzhuang, timesAlibaba, MIIT, iFlytek, technologyCheetah Mobile, languageArtificial Intelligence, intelligenceChina, AI, Daily Online}"
1,http://newsparliament.com/2020/02/27/children-with-autism-saw-their-learning-and-social-skills-boosted-after-playing-with-this-ai-robot/,2020-02-27,en,Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot – News Parliament,"\nChildren With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot – News Parliament\n \n\nSkip to content\n\t\t\tThursday, February 27, 2020\t\t\n\nLatest:\n\n\nMansplaining in conferences: How can we get him to forestall?\n\n\nDrax power station to cease burning coal in March 2021\n\n\nCoronavirus Could Explode in the U.S. Overnight Like it Did in Italy\n\n\nCoronavirus: Dettol sales surge as markets fall again\n\n\nLevi Strauss marks the next phase in cor...","Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot News Parliament Skip to contentThursday, February , Latest:Mansplaining in conferences: How can we get him to forestall?Drax power station to cease burning coal in March Coronavirus Could Explode in the U.S. Overnight Like it Did in ItalyCoronavirus: Dettol sales surge as markets fall againLevi Strauss marks the next phase in corporate paid leave policiesNews ParliamentGet the Real Story NewsTe...","{Science Robotics, Comments Author Recent Posts, admin0 About Us News Parliament, HRI, New Comprehensive Report, Trending News Mansplaining, Drax, Matari, ASD, Alex Jones0 BusinessNewsServicesTechnology New Study Examines Global Water Treatment Services, CAD International (ArborCAD, The Dow Chemical, Dettol, PolicyContact Us News Sections BusinessEntertainmentHealthcareNewsServicesTechnologyUncategorizedWorld - News Parliament, Alex Jones0 BusinessServicesTechnology Massive Growth of Arboris...","{Highlights, All Rights Reserved, HRI, Arborist Software Market By Top Key Players Forest Metrix, Prominent Players Dabur, Business News Services Technology New Study Examines Global, Coronavirus Could Explode, Comment Name, University, Parliament Skip, CAD International, ASD, TweetWhatsApp News Drax, Dow Chemical, SAR, ArborCAD, Social Skills Boosted, Industry Status, Downstream Oil, Aquatech International, Congress, Real, Minute New Orleans Hotel, Bus

In [ ]:
news_df.to_parquet("Vishal_NLP_data/news_df_companies.parquet")

In [ ]:
news_df = pd.read_parquet('Vishal_NLP_data/news_df_companies.parquet', engine='pyarrow')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1",padding=True)

model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

In [ ]:
prompt = "when tables opened up, the manager sat another party before us."

In [ ]:
tokens = tokenizer(news_df.iloc[3]['cleaned_text'], return_tensors="pt")

In [ ]:
len(tokens['input_ids'][0])

1309

In [8]:
news_df.iloc[3]['cleaned_text']

'Strategy Analytics: % of Smartphones Sold Globally in will be AI Powered Consumer Electronics Net Skip to contentConsumer Electronics NetPrimary MenuConsumer Electronics NetSearch for: HomeNewsStrategy Analytics: % of Smartphones Sold Globally in will be AI Powered News Strategy Analytics: % of Smartphones Sold Globally in will be AI Powered hours ago Artificial Intelligence Now Powers the Majority of SmartphonesBOSTON(BUSINESS WIRE)Strategy Analytics in a newly published report, Smartphones: Global Artificial Intelligence Technologies Forecast to , finds that on-device Artificial Intelligence (AI) is being rapidly implemented by smartphone vendors. AI is used in various functions inside smartphones such as intelligent power optimization, imaging, virtual assistants, and to enhance device performance. The report highlights the fact that AI has become the most important technology in modern smartphones. The push to add on-device AI computational power to enable Edge AI computing is a p

In [ ]:
def split_sentence(string):
    doc = nlp(string)
    sentences_tokens = []
    for sent in doc.sents:
        sentences_tokens.append(str(sent))
    return sentences_tokens

In [9]:
sentences = []
for doc in tqdm(nlp.pipe(
    texts = news_df.iloc[0:len(news_df)-1]['cleaned_text'].to_list(),
    n_process=os.cpu_count()
)):
    sents = []
    for sent in doc.sents:
        sents.append(str(sent))
    sentences.append(sents)

0it [00:00, ?it/s]c:\Users\yomaa\Documents\Environments\spacy\lib\site-packages\spacy\language.py:1566: UserWarning: [W114] Using multiprocessing with GPU models is not recommended and may lead to errors.
  warnings.warn(Warnings.W114)
0it [02:06, ?it/s]


KeyboardInterrupt: 

In [ ]:
news_df['sentences'] = sentences

In [ ]:
sentences

[['Artificial intelligence improves parking efficiency in Chinese cities',
  "- People's Daily OnlineHomeChina PoliticsForeign AffairsOpinionsVideo: We Are ChinaBusinessMilitaryWorldSocietyCultureTravelScienceSportsPhotoLanguagesChineseJapaneseFrenchSpanishRussianArabicKoreanGermanPortugueseThursday, March , Home>>Artificial intelligence improves parking efficiency in Chinese citiesBy LiuShiyao (People's Daily) :16, March , Photo taken on July , , shows a sign for electronic toll collection (ETC) newly set up at a roadside parking space on Yangzhuang road, Shijingshan district, Beijing.",
  'Some urban areas of the city started to use ETC system for roadside parking spaces since July , .',
  '(Peoples Daily Online/Li Wenming)Thanks to the application of an artificial intelligence (AI)-empowered roadside electronic toll collection (ETC) system, Chinas capital city Beijing has seen significant improvement in the efficiency of parking fee collection, turnover of roadside parking spots, or

In [ ]:
prediction_master = []
for i in tqdm(range(len(news_df))):
    string = news_df.iloc[i]['cleaned_text']
    sentences = split_sentence(string)
    sentences = ["[CLS] " + sent + " [SEP] AI [SEP] " for sent in sentences]
    predictions = []
    labels = ["Negative","Neutral","Positive"]
    n = 8
    for i in range(0, len(sentences)-n+1, n):
        tokens = tokenizer(sentences[i:i+n], return_tensors="pt",padding="max_length",max_length=256,truncation=True)
        outputs = model(**tokens)
        probs = F.softmax(outputs.logits,dim=0)
        probs = probs.detach().numpy()
        values = np.argmax(probs,axis=1)
        predictions = predictions + [labels[value] for value in values]
    length = len(predictions)
    ratios = [predictions.count("Negative")/length,predictions.count("Neutral")/length,predictions.count("Positive")/length]
    temp_labels = ["Negative","Positive"]
    if max(ratios) == ratios[1]:
        if abs(ratios[1]-ratios[0])<0.4 or abs(ratios[1]-ratios[2])<0.4:
            temp = [ratios[0],ratios[2]]
            prediction = temp_labels[np.argmax(temp)]
        else:
            prediction = "Neutral"
    else:
        prediction = labels[np.argmax(ratios)]
    prediction_master.append(prediction)
prediction_master

  0%|          | 48/200332 [02:51<198:28:37,  3.57s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:11                                                                                   │
│                                                                                                  │
│    8 │   n = 8                                                                                   │
│    9 │   for i in range(0, len(sentences)-n+1, n):                                               │
│   10 │   │   tokens = tokenizer(sentences[i:i+n], return_tensors="pt",padding="max_length",ma    │
│ ❱ 11 │   │   outputs = model(**tokens)                                                           │
│   12 │   │   probs = F.softmax(outputs.logits,dim=0)                                             │
│   13 │   │   probs = probs.detach().numpy()                                                      │
│   14 │   │   values = np.argmax(probs,axis=1)                                                    │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:1194 in _call_impl             │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:131 │
│ 9 in forward                                                                                     │
│                                                                                                  │
│   1316 │   │   │   inputs_embeds=inputs_embeds,                                                  │
│   1317 │   │   │   output_attentions=output_attentions,                                          │
│   1318 │   │   │   output_hidden_states=output_hidden_states,                                    │
│ ❱ 1319 │   │   │   return_dict=return_dict,                                                      │
│   1320 │   │   )                                                                                 │
│   1321 │   │                                                                                     │
│   1322 │   │   encoder_layer = outputs[0]                                                        │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:1194 in _call_impl             │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks